In [1]:
import pandas as pd

In [2]:
path = "C:\\Users\\visha\\Downloads\\Subtitle_Seachengine\\Subtitle_Search_Engine-main\\Subtitle_Search_Engine-main\\Data\\Datasubtitles_extracted.parquet"
df = pd.read_parquet(path)

In [3]:
df.head()

,subtitle_id,subtitle_name,content
3536,9399021,kyuketsuki.sugu.shinu.s01.e05.the.longest.day....,stray cat from the neighborhoodNgot lost in th...
2012,9248544,the.real.bling.ring.hollywood.heist.s01.e02.fa...,n during those years and so my way of being in...
24613,9354469,beverly.hills.90210.s03.e14.wild.horses.(1992)...,to you about what happened last night Theres ...
12239,9359648,according.to.jim.s03.e06.abcs.and.123s.(2003)....,you cant read Yeah Read this bucko You know yo...
5295,9232597,paappan.(2022).eng.1cd,ases yesterday Its time So we havent been able...


In [5]:
df.reset_index(drop=True)
df.head()

,subtitle_id,subtitle_name,content
3536,9399021,kyuketsuki.sugu.shinu.s01.e05.the.longest.day....,stray cat from the neighborhoodNgot lost in th...
2012,9248544,the.real.bling.ring.hollywood.heist.s01.e02.fa...,n during those years and so my way of being in...
24613,9354469,beverly.hills.90210.s03.e14.wild.horses.(1992)...,to you about what happened last night Theres ...
12239,9359648,according.to.jim.s03.e06.abcs.and.123s.(2003)....,you cant read Yeah Read this bucko You know yo...
5295,9232597,paappan.(2022).eng.1cd,ases yesterday Its time So we havent been able...


In [6]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 468111 entries, 3536 to 371
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   subtitle_id    468111 non-null  object
 1   subtitle_name  468111 non-null  object
 2   content        468111 non-null  object
dtypes: object(3)
memory usage: 331.0 MB


# Embeddings

In [8]:
import chromadb

In [10]:
path = "C:\\Users\\visha\\Downloads\\Subtitle_Seachengine\\Subtitle_Search_Engine-main\\Subtitle_Search_Engine-main\\db"

In [13]:
client = chromadb.PersistentClient(path=path)

In [14]:
client.heartbeat()

1714065793678832300

In [15]:
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer,util

C:\Users\visha\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
sentence_transformer = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

In [17]:
collection = client.create_collection(
        name="subtitle_sem",
        metadata={"hnsw:space": "cosine"},
        embedding_function=sentence_transformer
    )

In [18]:
new_df = df.sample(frac=0.20, random_state=42)
new_df.reset_index(drop=True, inplace=True)

In [19]:
import re
new_df['subtitle_name'] = new_df['subtitle_name'].apply(lambda x: re.sub('\.', ' ', x))

In [20]:
new_df['content'] = new_df['content'].apply(lambda x: x.lower())

In [21]:
content = new_df['content'].tolist()
metadatas = [{'subtitle_name': name, 'subtitle_id': id} for name, id in zip(new_df['subtitle_name'], new_df['subtitle_id'])]
ids = [str(i) for i in range(len(new_df))]

In [22]:
batch_size = 5000
num_batches = (len(content) + batch_size - 1) // batch_size

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(content))

    batch_content = content[start_idx:end_idx]
    batch_metadatas = metadatas[start_idx:end_idx]
    batch_ids = ids[start_idx:end_idx]

    collection.add(
        documents=batch_content,
        metadatas=batch_metadatas,
        ids=batch_ids
    )




# Let's test the code

In [23]:
query = 'Life is like riding a bicycle. To keep your balance, you must keep moving.'

In [24]:
result = collection.query(
    query_texts = query,
    include=["metadatas", 'distances'],
    n_results=5
)

In [26]:
ids = result['ids'][0]
distances = result['distances'][0]
metadatas = result['metadatas'][0]
zipped_data = zip(ids, distances, metadatas)
sorted_data = sorted(zipped_data, key=lambda x: x[1], reverse=True)
for _, distance, metadata in sorted_data:
    subtitle_name = metadata['subtitle_name']
    print(f"Subtitle Name: {subtitle_name.upper()}")

Subtitle Name: ONE NATION UNDER STRESS (2019) ENG 1CD
Subtitle Name: LES TROIS SOEURS (2015) ENG 1CD
Subtitle Name: BODY CAM (2020) ENG 1CD
Subtitle Name: SATURDAY NIGHT LIVE S02 E01 LILY TOMLINJAMES TAYLOR (1976) ENG 1CD
Subtitle Name: THE XFILES S02 E08 ONE BREATH (1994) ENG 1CD
